# Import Libraries

In [13]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle
from sklearn.preprocessing import scale
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.models import load_model
from keras import metrics
from keras.regularizers import l2
import models.nnmodels as nn
import utils.data as data

import gc
gc.enable()
seed = 0
np.random.seed(seed)
%matplotlib inline

# Load DEAP pre-processed data

In [14]:
print 'Loading DEAP dataset...'
subject_data_folds, valence_folds, arousal_folds = data.load_deap(32, 2)
print 'DEAP dataset is loaded.\n'

print 5*'#' + ' Dataset representation for each subject ' + 5*'#'
print '\nThere are {} subjects in the dataset.\n'.format(len(subject_data_folds))
print '(Trail/Video, Channel, Readings) : ', subject_data_folds[0].shape
print '(Trail/Video, label (valence or arousal)) : ', arousal_folds[0].shape

Loading DEAP dataset...
DEAP dataset is loaded.

##### Dataset representation for each subject #####

There are 32 subjects in the dataset.

(Trail/Video, Channel, Readings) :  (40, 40, 8064)
(Trail/Video, label (valence or arousal)) :  (40, 2)


# Normalize

In [15]:
# Concatenate the data
X = np.concatenate(subject_data_folds)
# Reshape
(trials, channels, readings) = X.shape
X_reshaped = X.reshape(trials, channels*readings)

# Normalize
X_reshaped_normalized = scale(X_reshaped)

X_normalized =  X_reshaped_normalized.reshape(trials, channels, readings)

Normalized_data = np.split(X_normalized, 32)

In [16]:
# Split each sample into 10 sample (400 experiment per subject) 


# Select last 30 seconds of only EGG channels
first_4_readings_removed = []
for fold in Normalized_data:
    first_4_readings_removed.append(fold[:,:,4:])
    
# Sanity check the shape 
print ('Shape of each subject data: {}'.format(first_4_readings_removed[0].shape))

augmented_data_folds = []
for fold in first_4_readings_removed:
    
    subject_splited = np.split(fold, 10, axis=-1)
    augmented_subject = np.concatenate(subject_splited)
    
    augmented_data_folds.append(augmented_subject)

# Produce the labels for segments
augmented_valence_labels = []
for l in valence_folds:
    new_l = l.repeat(10, axis=0)
    augmented_valence_labels.append(new_l)

# Sanity check the shape of subject and labels
print ('Augmented subject has the shape of {}'.format(augmented_data_folds[0].shape))
print ('Augmented valence labels has the shape of {}'.format(augmented_valence_labels[0].shape))

Shape of each subject data: (40, 40, 8060)
Augmented subject has the shape of (400, 40, 806)
Augmented valence labels has the shape of (400, 2)


# Reduce dimension

In [17]:
# Reduce the features dimension
print 'Mapping features into Gaussian with mean zero and unit variance...'
Normalized_data_reduced = data.normalize_features(augmented_data_folds, pre_norm=False)
print 'Normalization done.'

Mapping features into Gaussian with mean zero and unit variance...
Normalization done.


In [18]:
Normalized_data_reduced[0].shape

(400, 40, 29)

In [19]:
plt.plot(Normalized_data_reduced[0][1,:,:])


with open('Normalized_data_reduced.pickle', 'wb') as handle:
    pickle.dump(Normalized_data_reduced, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('augmented_valence_labels.pickle', 'wb') as handle:
    pickle.dump(augmented_valence_labels, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.models import load_model
from keras import metrics
from sklearn.preprocessing import scale
from keras.regularizers import l2
import models.nnmodels as nn
import utils.data as data

import gc
gc.enable()
seed = 0
np.random.seed(seed)
%matplotlib inline


with open('Normalized_data_reduced.pickle', 'rb') as handle:
    Normalized_data_reduced = pickle.load(handle)
    
with open('augmented_valence_labels.pickle', 'rb') as handle:
    augmented_valence_labels = pickle.load(handle)

In [20]:
um_exp = 40
num_classes = 2
batch_size = 50
epochs = 50
num_subjects = 32


accuracies= []

print '##### Two Classes Classification for Valence emotion Using Convolutional Neural Network #####\n'
   

for s in np.arange(0, num_subjects):
    
    
    X_val = Normalized_data_reduced[s]
    X_val = np.expand_dims(X_val, axis=1)
    y_val = augmented_valence_labels[s]
    
    X_remaining_folds = Normalized_data_reduced[:s] + Normalized_data_reduced[s+1:]
    y_remaining_folds = augmented_valence_labels[:s] + augmented_valence_labels[s+1:]
    
    X_train = np.concatenate(X_remaining_folds)
    X_train = np.expand_dims(X_train, axis=1)
    
    y_train = np.concatenate(y_remaining_folds)
    
    # To test major vote predictions at the end
    y_test = valence_folds[s]
    
    
    key = 's'+str(s+1).zfill(2)+'_valence(2classes)'
    
    
    # Valence classification model for two classes    
    model = nn.CNN(num_classes=num_classes, emotion='valence', input_shape=X_train.shape[1:])
    
    
    # checkpoint
    filepath="../saved_models/best_CNN_model_for_{}.h5".format(key)
    
    print 'Training model to test on subject {} ...\n'.format(str(s+1).zfill(2))
    
    checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy',
                                verbose=1, save_best_only=True, mode='max', period=20)
    
    callbacks_list = [checkpoint]
    
    
    hist = model.fit(X_train, y_train, 
             batch_size=batch_size,
             epochs=epochs,
             shuffle=True,
             verbose=1,
             validation_data=(X_val, y_val), callbacks=callbacks_list)

    
    
    model.load_weights(filepath)
    
    scores = model.evaluate(x=X_val, y=y_val)
    
    y_val_pred = model.predict(X_val)
    
    pred_list = np.split(predictions, 40)
    
    y_test_pred_list = []
    for pred in pred_list:
        
        class_zero = np.sum(pred[:,0])
        class_one = np.sum(pred[:,1])
        
        if class_zero >class_one:
            y_test_list.append(np.array([[1,0]]))
        elif class_zero <= class_one:
            y_test_list.append(np.array([[1,0]]))
            
    y_test_pred = np.concatenate(y_test_pred_list)
    
    truth_table = y_test_pred == y_test
        
    acc = 100*np.sum(truth_table)/len(truth_table)
        
        
    
    
    
    print '#### Result of CNN model for subject {} #####\n'.format(str(s+1).zfill(2))
    print "Validation Loss: {} , Validation Accuracy : {}%".format(scores[0], scores[1]*100)
    
    print 'Major voting accuracy is: {}%'.format(acc)
    
    

    accuracies.append(acc)
    # summarize history for accuracy
    plt.plot(hist.history['categorical_accuracy'])
    plt.plot(hist.history['val_categorical_accuracy'])
    plt.title('Subject ' +str(s+1).zfill(2)+ ' CNN model accuracy for Valence')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    
    # summarize history for loss
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('Subject ' +str(s+1).zfill(2)+ ' CNN model loss for Valence')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    del model

##### Two Classes Classification for Valence emotion Using Convolutional Neural Network #####

Training model to test on subject 01 ...

Train on 12400 samples, validate on 400 samples
Epoch 1/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.6824 - categorical_accuracy: 0.5509 - val_loss: 0.7018 - val_categorical_accuracy: 0.4750
Epoch 2/250
12400/12400 [==============================] - 39s 3ms/step - loss: 0.6447 - categorical_accuracy: 0.6270 - val_loss: 0.6801 - val_categorical_accuracy: 0.5325
Epoch 3/250
12400/12400 [==============================] - 39s 3ms/step - loss: 0.6213 - categorical_accuracy: 0.6756 - val_loss: 0.6725 - val_categorical_accuracy: 0.6400
Epoch 4/250
12400/12400 [==============================] - 39s 3ms/step - loss: 0.6003 - categorical_accuracy: 0.7037 - val_loss: 0.6648 - val_categorical_accuracy: 0.7100
Epoch 5/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.5861 - categorical_accuracy: 0.7170 - val_los

12400/12400 [==============================] - 40s 3ms/step - loss: 0.5193 - categorical_accuracy: 0.7613 - val_loss: 0.7094 - val_categorical_accuracy: 0.7275
Epoch 45/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.5182 - categorical_accuracy: 0.7586 - val_loss: 0.7157 - val_categorical_accuracy: 0.7325
Epoch 46/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.5196 - categorical_accuracy: 0.7606 - val_loss: 0.7158 - val_categorical_accuracy: 0.7325
Epoch 47/250
12400/12400 [==============================] - 39s 3ms/step - loss: 0.5195 - categorical_accuracy: 0.7624 - val_loss: 0.7123 - val_categorical_accuracy: 0.7275
Epoch 48/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.5185 - categorical_accuracy: 0.7636 - val_loss: 0.7105 - val_categorical_accuracy: 0.7275
Epoch 49/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.5196 - categorical_accuracy: 0.7602 - val_loss: 0.7167 - val_cate

12400/12400 [==============================] - 40s 3ms/step - loss: 0.5092 - categorical_accuracy: 0.7684 - val_loss: 0.7155 - val_categorical_accuracy: 0.7325
Epoch 90/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.5086 - categorical_accuracy: 0.7699 - val_loss: 0.7168 - val_categorical_accuracy: 0.7350
Epoch 91/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.5077 - categorical_accuracy: 0.7689 - val_loss: 0.7142 - val_categorical_accuracy: 0.7350
Epoch 92/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.5086 - categorical_accuracy: 0.7698 - val_loss: 0.7107 - val_categorical_accuracy: 0.7350
Epoch 93/250
12400/12400 [==============================] - 39s 3ms/step - loss: 0.5075 - categorical_accuracy: 0.7716 - val_loss: 0.7105 - val_categorical_accuracy: 0.7350
Epoch 94/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.5071 - categorical_accuracy: 0.7696 - val_loss: 0.7183 - val_cate

Epoch 134/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.4987 - categorical_accuracy: 0.7764 - val_loss: 0.7196 - val_categorical_accuracy: 0.7250
Epoch 135/250
12400/12400 [==============================] - 39s 3ms/step - loss: 0.5002 - categorical_accuracy: 0.7745 - val_loss: 0.7159 - val_categorical_accuracy: 0.7300
Epoch 136/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.5002 - categorical_accuracy: 0.7756 - val_loss: 0.7205 - val_categorical_accuracy: 0.7225
Epoch 137/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.5001 - categorical_accuracy: 0.7744 - val_loss: 0.7123 - val_categorical_accuracy: 0.7250
Epoch 138/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.5007 - categorical_accuracy: 0.7752 - val_loss: 0.7181 - val_categorical_accuracy: 0.7275
Epoch 139/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.5008 - categorical_accuracy: 0.7746 - val_loss

12400/12400 [==============================] - 39s 3ms/step - loss: 0.4928 - categorical_accuracy: 0.7777 - val_loss: 0.7162 - val_categorical_accuracy: 0.7225
Epoch 180/250
12400/12400 [==============================] - 39s 3ms/step - loss: 0.4919 - categorical_accuracy: 0.7798 - val_loss: 0.7169 - val_categorical_accuracy: 0.7200
Epoch 181/250
12400/12400 [==============================] - 39s 3ms/step - loss: 0.4932 - categorical_accuracy: 0.7768 - val_loss: 0.7249 - val_categorical_accuracy: 0.7125
Epoch 182/250
12400/12400 [==============================] - 39s 3ms/step - loss: 0.4947 - categorical_accuracy: 0.7774 - val_loss: 0.7184 - val_categorical_accuracy: 0.7200
Epoch 183/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.4928 - categorical_accuracy: 0.7786 - val_loss: 0.7162 - val_categorical_accuracy: 0.7225
Epoch 184/250
12400/12400 [==============================] - 40s 3ms/step - loss: 0.4933 - categorical_accuracy: 0.7788 - val_loss: 0.7150 - val

KeyboardInterrupt: 

In [ ]:
min_acc_subject_idx, max_acc_subject_idx = np.argmin(accuracies), np.argmax(accuracies)

avg_acc =  reduce(lambda x, y: x + y, accuracies) / len(accuracies)

print 5*'#' + ' Accuracies per subject ' + 5*'#'

print 'Subject {} has the minimum accuracy : {}%'.format(min_acc_subject_idx + 1, min(accuracies)*100)
print 'Subject {} has the maximum accuracy : {}%'.format(max_acc_subject_idx + 1, max(accuracies)*100)
print 'Average accuracy : {}%'.format(avg_acc*100)

for subj in enumerate(accuracies):
    print 'Accuracy of Subject {} : {}%'.format(subj[0]+1, subj[1]*100)